In [ ]:
raise

# this note book cannot be run

```cli
psql -U postgres
```

```cli
postgres=# CREATE DATABASE ztm;
postgres=# exit
```

```cli
psql -U postgres ztm
ztm=# \conninfo
```

output
```
You are connected to database "ztm" as user "postgres" on host "localhost" (address "::1") at port "5432".
```

```cli
ztm=# CREATE EXTENSION IF NOT EXISTS "UUID-OSSP";
```

In [ ]:
CREATE TABLE student (
  student_id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
  first_name VARCHAR(255) NOT NULL,
  last_name VARCHAR(255) NOT NULL,
  email VARCHAR(255) NOT NULL,
  date_of_birth DATE NOT NULL 
);

```
ztm=# \dt
```

output
```
          List of relations
 Schema |  Name   | Type  |  Owner
--------+---------+-------+----------
 public | student | table | postgres
(1 row)
```

```
ztm=# \d student
```

output
```
                               Table "public.student"
    Column     |          Type          | Collation | Nullable |      Default
---------------+------------------------+-----------+----------+--------------------
 strudent_id   | uuid                   |           | not null | uuid_generate_v4()
 first_name    | character varying(255) |           | not null |
 last_name     | character varying(255) |           | not null |
 email         | character varying(255) |           | not null |
 date_of_birth | date                   |           | not null |
Indexes:
    "student_pkey" PRIMARY KEY, btree (strudent_id)
```

<img src="./assets/07-01-entity-relationship.png" />

In [ ]:
-- custom data types

CREATE DOMAIN Rating SMALLINT
    CHECK (VALUE > 0 AND VALUE <= 5);

CREATE TYPE Feedback AS (
    student_id UUID,
    rating Rating,
    feedback TEXT
);

In [ ]:
CREATE TABLE subject (
  subject_id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
  subject TEXT NOT NULL,
  description TEXT
);

In [ ]:
CREATE TABLE teacher (
  teacher_id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
  first_name VARCHAR(255) NOT NULL,
  last_name VARCHAR(255) NOT NULL,
  email TEXT,
  date_of_birth DATE NOT NULL 
);

In [ ]:
ALTER TABLE student
ADD COLUMN email TEXT;

In [ ]:
-- name is duplitcate with SQL keyword (syntax). So, we put in "name"

CREATE TABLE course (
  course_id UUID PRIMARY KEY DEFAULT uuid_generate_v4(),
  "name" TEXT NOT NULL,
  description TEXT,
  subject_id UUID REFERENCES subject(subject_id),
  teacher_id UUID REFERENCES teacher(teacher_id),
  feedback feedback[]
);

In [ ]:
CREATE TABLE enrollment (
  course_id UUID REFERENCES course(course_id),
  student_id UUID REFERENCES student(student_id),
  enrollment_date DATE NOT NULL,
  CONSTRAINT pk_emrollment PRIMARY KEY (course_id, student_id)
);

## Adding data

In [ ]:
INSERT INTO student(
  first_name,
  last_name,
  email,
  date_of_birth
) VALUES (
  'Mo',
  'Binni',
  'mo@binni.io',
  '1992-11-13'::DATE
);

In [ ]:
INSERT INTO teacher(
  first_name,
  last_name,
  email,
  date_of_birth
) VALUES (
  'Mo',
  'Binni',
  'mo@binni.io',
  '1992-11-13'::DATE
);

In [ ]:
INSERT INTO subject (
  subject,
  description
) VALUES (
  'SQL Zero to Mastery',
  'The art of SQL Mastery'
);

DELETE FROM subject WHERE subject = 'SQL Zero to Mastery';

In [ ]:
INSERT INTO course (
  "name",
  description
) VALUES (
  'SQL Zero to Master',
  'The #1 resource for SQL mastery'
);

In [ ]:
UPDATE course
SET subject_id = 'cefa2138-12ef-427c-babc-c51e51b9af68'
WHERE subject_id IS NULL;

In [ ]:
ALTER TABLE course ALTER COLUMN subject_id SET NOT NULL;

In [ ]:
UPDATE course
SET teacher_id = '023e8e9c-0744-4fe5-a613-0b3da36a1fbc'
WHERE teacher_id IS NULL;

In [ ]:
ALTER TABLE course ALTER COLUMN teacher_id SET NOT NULL;

In [ ]:
INSERT INTO enrollment (student_id, course_id, enrollment_date) VALUES (
  '467ddbcc-afbc-43a4-9f94-b3b07c33b713',
  'ecd2333c-7d15-4466-8ca3-9bce61f40326',
  NOW()::DATE
);

In [ ]:
UPDATE course
SET feedback = array_append(
  feedback,
  ROW(
    '467ddbcc-afbc-43a4-9f94-b3b07c33b713',
    5,
    'Great course!'
  )::feedback
)
WHERE course_id = 'ecd2333c-7d15-4466-8ca3-9bce61f40326'

<img src="./assets/07-03-er-final.png" />

In [ ]:
CREATE TABLE feedback (
  student_id UUID NOT NULL REFERENCES student(student_id),
  course_id UUID NOT NULL REFERENCES course(course_id),
  feedback TEXT,
  rating rating,
  CONSTRAINT pk_feedback PRIMARY KEY (student_id, course_id)
);

In [ ]:
INSERT INTO feedback (
  student_id, course_id, feedback, rating
) VALUES (
  '467ddbcc-afbc-43a4-9f94-b3b07c33b713', 
  'ecd2333c-7d15-4466-8ca3-9bce61f40326',
  'Well, this was great!',
  5
);